In [38]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the ClinicalBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=3)



Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

In [39]:
# Define example clinical text
text = "The patient was diagnosed with diabetes and hypertension."

# Tokenize the text and convert to input format
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([token_ids])
attention_mask = torch.tensor([[1] * len(token_ids)])
inputs = {"input_ids": input_ids, "attention_mask": attention_mask}




In [40]:
# Pass the input through the model and get the predicted labels
outputs = model(**inputs)
predicted_labels = torch.argmax(outputs.logits, axis=2)[0]

# Map the predicted labels back to the original tokenization and extract entities
entities = []
entity_labels = []
current_entity = ""
current_label = None
for i, label in enumerate(predicted_labels):
    if label == 1:
        if current_entity:
            entities.append(current_entity)
            entity_labels.append(current_label)
            current_entity = ""
            current_label = None
        current_entity = tokens[i]
        current_label = label
    elif i > 0 and label == 2 and predicted_labels[i-1] == 1:
        current_entity += f" {tokens[i]}"
    elif label == 2:
        if current_entity:
            entities.append(current_entity)
            entity_labels.append(current_label)
            current_entity = ""
            current_label = None



In [42]:
# If an entity is still being processed at the end of the text, append it
if current_entity:
    entities.append(current_entity)
    entity_labels.append(current_label)

# Print the predicted entities and their labels
for entity, label in zip(entities, entity_labels):
    print(f"{entity}: {label}")

the: 1
patient: 1
was: 1
with: 1
and: 1
##tens: 1
##ion: 1
##ion: 1
